<a href="https://colab.research.google.com/github/nyquist/aoe2de/blob/main/AOE2DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config Replay

In [85]:
game_id = "357732557"

#Environment Setup

In [86]:
! [[ -d "aoe2de" ]] && (cd aoe2de && git pull) || git clone https://github.com/nyquist/aoe2de.git
! ls -la ./aoe2de/utils

Already up to date.
total 12
drwxr-xr-x 2 root root 4096 Dec 14 14:36 .
drwxr-xr-x 5 root root 4096 Dec 14 16:59 ..
-rw-r--r-- 1 root root   22 Dec 14 14:36 test.py


In [87]:
import os

def list_files_with_dynamic_sizes(path="."):
    """Lists all files and their sizes with dynamic units (B, kB, MB, etc.) in the given directory.

    Args:
        path (str, optional): The directory path to list files from. Defaults to "." (current directory).
    """
    for entry in os.scandir(path):
        if entry.is_file():
            file_size = os.path.getsize(entry.path)
            unit_index = 0
            units = ["B", "kB", "MB", "GB", "TB"]
            while file_size >= 1024 and unit_index < len(units) - 1:
                file_size /= 1024
                unit_index += 1
            print(f"{entry.name}: {file_size:.2f} {units[unit_index]}")

# Call the function to list files and sizes with dynamic units in the current directory


# Download record

In [88]:
import requests

replay_url = f"https://aoe.ms/replay/?gameId={game_id}&profileId=18446743"  # Replace with the actual URL you copied

response = requests.get(replay_url)
response.raise_for_status()  # Raise an exception for bad responses
print (response.status_code)
content_disposition = response.headers['Content-Disposition']
import re
match = re.search(r'filename="?([^";]+)"?', content_disposition)
if match:
    file_name = match.group(1)
    print(f"Extracted filename: {file_name}")
    with open(file_name, "wb") as f:
      f.write(response.content)
else:
    print("Filename not found in Content-Disposition header.")


print(f"Replay file downloaded as {file_name}")
list_files_with_dynamic_sizes()





200
Extracted filename: AgeIIDE_Replay_357732557.zip
Replay file downloaded as AgeIIDE_Replay_357732557.zip
AgeIIDE_Replay_357732557.zip: 1.37 MB
AgeIIDE_Replay_357732557.aoe2record: 3.09 MB
AgeIIDE_Replay_357732557.aoe2record.json: 16.03 MB


### Unzip record file

In [89]:
zip_file_name = file_name

import zipfile

try:
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(".")  # Extract all files to the current directory
    print(f"Successfully extracted files from {zip_file_name}")

    # Find the extracted replay file (assuming it's an .mgz file)
    extracted_file_name = None
    for filename in os.listdir("."):
        print (filename)
        if filename.endswith(".aoe2record"):
            extracted_file_name = filename
            break

    if extracted_file_name:
        # Process the extracted replay file
        print ("Extracted file:", extracted_file_name)
    else:
        print("Error: Could not find extracted replay file (.mgz).")

except zipfile.BadZipFile:
    print(f"Error: {zip_file_name} is not a valid zip file.")
except FileNotFoundError:
    print(f"Error: {zip_file_name} not found.")
except RuntimeError as e:
    if "could not parse" in str(e):
        print(f"Error parsing the replay file: {e}")
        print("The replay file might be corrupted or in an unsupported format.")
        print("Please try re-downloading the replay file or using a different one.")
    else:
        raise  # Re-raise the exception if it's not related to parsing

Successfully extracted files from AgeIIDE_Replay_357732557.zip
.config
AgeIIDE_Replay_357732557.zip
AgeIIDE_Replay_357732557.aoe2record
Extracted file: AgeIIDE_Replay_357732557.aoe2record


## Convert file to json

In [90]:
import json
! pip install mgz
from mgz.model import parse_match, serialize
f = extracted_file_name
outFile=f"{extracted_file_name}.json"
print (f)
with open(f, 'rb') as h:
    match = parse_match(h)
    # print(json.dumps(serialize(match), indent=2))
    json_object=json.dumps(serialize(match), indent=2)
    with open(outFile, "w") as o:
        o.write(json_object)

list_files_with_dynamic_sizes()

AgeIIDE_Replay_357732557.aoe2record
AgeIIDE_Replay_357732557.zip: 1.37 MB
AgeIIDE_Replay_357732557.aoe2record: 3.09 MB
AgeIIDE_Replay_357732557.aoe2record.json: 16.03 MB


# Analysis

In [91]:
with open(outFile, 'r') as json_file:
    json_data = json.load(json_file)

## Buildings

In [92]:
#! head aoe2de/mtmExtractBuildingsfromJson.py

from aoe2de.mtmExtractBuildingsfromJson import BuildingsAnalyzer

buildings = BuildingsAnalyzer(outFile)

risk 1
email_to_mihai 3
nyquist 5
gabisan 7
Donald 2
Fonke 4
jempy1990 6
Kwak 8
['risk', 'Donald', 'email_to_mihai', 'Fonke', 'nyquist', 'jempy1990', 'gabisan', 'Kwak']
[{}, {}, {}, {}, {}, {}, {}, {}]
+----------------+------+--------+----------------+-------+---------+-----------+---------+------+
|    Cladire     | risk | Donald | email_to_mihai | Fonke | nyquist | jempy1990 | gabisan | Kwak |
+----------------+------+--------+----------------+-------+---------+-----------+---------+------+
|     House      |  22  |   51   |       31       |   30  |    14   |     29    |    27   |  42  |
| Palisade Wall  |  2   |   1    |       2        |   0   |    3    |     1     |    1    |  2   |
|  Lumber Camp   |  1   |   6    |       3        |   3   |    5    |     4     |    2    |  3   |
|      Mill      |  1   |   1    |       1        |   1   |    2    |     1     |    1    |  2   |
|  Mining Camp   |  2   |   4    |       4        |   4   |    3    |     4     |    5    |  3   |
|     